In [4]:
# author : shadril238

# Imports
!ollama pull llama3.2
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import ollama

pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠴ pulling manifest ⠧ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠇ pulling manifest ⠇ pulling manifest 
pulling dde5aa3fc5ff... 100% ▕████████████████▏ 2.0 GB                         
pulling 966de95ca8a6... 100% ▕████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da... 100% ▕████████████████▏ 7.7 KB                         
pulling a70ff7e570d9... 100% ▕████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B                         
pulling 34bb5ab01051... 100% ▕████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [5]:
# Constants
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [7]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [10]:
bs23 = Website(url='https://brainstation-23.com/?bs')
bs23.links

['#content',
 'https://brainstation-23.com/',
 'https://brainstation-23.com/',
 'https://brainstation-23.com/services/',
 'https://brainstation-23.com/field-force-automation/',
 'https://brainstation-23.com/adobe-experience-manager-aem/',
 'https://brainstation-23.com/banking-solution/',
 'https://brainstation-23.com/mobile-app-development/',
 '#',
 'https://brainstation-23.com/augmented-reality/',
 'https://brainstation-23.com/virtual-reality/',
 'https://brainstation-23.com/?page_id=22946',
 'https://brainstation-23.com/game-studio/',
 'https://cloud-23.com/',
 'http://www.erp-23.com/',
 'https://www.nop-station.com',
 'https://brainstation-23.com/sharepoint-services/',
 'https://brainstation-23.com/data-science-business-intelligence/',
 'https://brainstation-23.com/ml-ai/',
 'https://brainstation-23.com/software-testing-as-service/',
 'https://brainstation-23.com/industries/',
 'https://brainstation-23.com/fintech-solutions/',
 'https://brainstation-23.com/healthcare/',
 'https://br

In [11]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [12]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [13]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [15]:
print(get_links_user_prompt(bs23))

Here is the list of links on the website of https://brainstation-23.com/?bs - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
#content
https://brainstation-23.com/
https://brainstation-23.com/
https://brainstation-23.com/services/
https://brainstation-23.com/field-force-automation/
https://brainstation-23.com/adobe-experience-manager-aem/
https://brainstation-23.com/banking-solution/
https://brainstation-23.com/mobile-app-development/
#
https://brainstation-23.com/augmented-reality/
https://brainstation-23.com/virtual-reality/
https://brainstation-23.com/?page_id=22946
https://brainstation-23.com/game-studio/
https://cloud-23.com/
http://www.erp-23.com/
https://www.nop-station.com
https://brainstation-23.com/sharepoint-services/
https://brainstation-23.com/data-science-business-intelligence/
https://brainstati

In [16]:
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [17]:
def get_links(url):
    website = Website(url)
    response = ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [19]:
get_links('https://brainstation-23.com/?bs')

{'links': [{'type': 'About page', 'url': 'https://brainstation-23.com/about/'},
  {'type': 'Company/Brain Station PLC',
   'url': 'https://brainstation-23.com/brain-station-23-plc/'},
  {'type': 'Services page', 'url': 'https://brainstation-23.com/services/'}]}

In [20]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [21]:
print(get_all_details("https://brainstation-23.com/?bs"))

Found links: {'links': [{'type': 'about page', 'url': 'https://brainstation-23.com/about/'}, {'type': 'services page', 'url': 'https://brainstation-23.com/services/'}, {'type': 'industries page', 'url': 'https://brainstation-23.com/industries/'}, {'type': 'case studies page', 'url': 'https://brainstation-23.com/case-studies/'}, {'type': 'blog page', 'url': 'https://brainstation-23.com/blog/'}, {'type': 'contact page', 'url': 'https://brainstation-23.com/contact/'}, {'type': 'partners page', 'url': 'https://brainstation-23.com/partners/'}]}
Landing page:
Webpage Title:
Brain Station 23 | Professionals @ Your Service
Webpage Contents:
Skip to content
Home
Services
Menu Toggle
Field Force Automation
Adobe Experience Manager
Banking Solution
Mobile App Development
AR-VR Solution
Menu Toggle
Augmented Reality
Virtual Reality
3D Modeling
Game Studio
Cloud & Cyber Security
ERP
E-commerce Solution
SharePoint Services
Data Science & Business Intelligence
ML & AI
Software Testing as Service
Indu

In [22]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [23]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [24]:
get_brochure_user_prompt("Brain Station 23", "https://brainstation-23.com/?bs")

Found links: {'links': [{'type': 'about page', 'url': 'https://brainstation-23.com/about/'}, {'type': 'company page', 'url': 'https://brainstation-23.com/brain-station-23-plc/'}, {'type': 'contact page', 'url': 'https://brainstation-23.com/contact/'}, {'type': 'Careers/Jobs page', 'url': 'https://erp.bs-23.com/jobs'}, {'type': 'Blog', 'url': 'https://medium.com/brain-station-23'}, {'type': 'Network page', 'url': 'https://www.linkedin.com/company/brain-station-23'}, {'type': 'Social Media', 'url': 'https://www.facebook.com/brainstation23'}, {'type': 'Twitter handle', 'url': 'https://twitter.com/BrainStation23'}]}


'You are looking at a company called: Brain Station 23\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nBrain Station 23 | Professionals @ Your Service\nWebpage Contents:\nSkip to content\nHome\nServices\nMenu Toggle\nField Force Automation\nAdobe Experience Manager\nBanking Solution\nMobile App Development\nAR-VR Solution\nMenu Toggle\nAugmented Reality\nVirtual Reality\n3D Modeling\nGame Studio\nCloud & Cyber Security\nERP\nE-commerce Solution\nSharePoint Services\nData Science & Business Intelligence\nML & AI\nSoftware Testing as Service\nIndustries\nMenu Toggle\nFintech\nHealthcare\nTelecom\nReal Estate\nSoftware/ITES\ne-Commerce\nEducation\nRetail\nAutomotive\nStartup\nCase Studies\nBlog\nAbout Us\nMenu Toggle\nAbout Us\nMedia\nOur Partners\nCareer\nInvestors\nContact Us\nMain Menu\nHome\nServices\nMenu Toggle\nField Force Automation\nAdobe Experience Mana

In [25]:
def create_brochure(company_name, url):
    response = ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [26]:
create_brochure("Brain Station 23", "https://brainstation-23.com/?bs")

Found links: {'links': [{'type': 'About page', 'url': 'https://brainstation-23.com/about/'}, {'type': 'Company page', 'url': 'https://brainstation-23.com/'}, {'type': 'Careers/Jobs page', 'url': 'https://erp.bs-23.com/jobs'}, {'type': 'Media/Media 3 page', 'url': 'https://brainstation-23.com/media-3/'}, {'type': 'Blog', 'url': 'https://brainstation-23.com/blog/'}, {'type': 'Case studies', 'url': 'https://brainstation-23.com/case-studies/'}]}


# Brain Station 23 Brochure

## Mission Statement
To deliver tailored digital transformation solutions and advanced IT services that empower businesses to thrive in an ever-evolving technological landscape.

## About Us
At Brain Station 23, we are professionals @ your service. With over 17 years of experience serving enterprises and SMEs across the globe, we have established ourselves as a trusted partner for innovative IT solutions.

Our global presence spans three continents, with offices in Dubai (UAE), Springfield, VA, USA, and Kuala Lumpur, Malaysia. Our team of experts comprises front-end developers, full-stack developers, UI/UX designers, product owners, scrum masters, DevOps engineers, and many more.

## Services
We offer a comprehensive range of IT services that cater to the diverse needs of our clients:

*   **Field Force Automation**: Automate field operations to enhance efficiency and productivity.
*   **Adobe Experience Manager**: Enhance customer experiences with robust digital engagement platforms.
*   **Banking Solution**: Leverage cutting-edge technology to drive banking services and expand online capabilities.
*   **Mobile App Development**: Develop innovative mobile apps that drive user experience and loyalty.
*   **AR-VR Solution**: Transform experiences with immersive augmented reality and virtual reality solutions.

We understand the unique challenges of various industries, including:

*   Fintech
*   Healthcare
*   Telecom
*   Real Estate
*   Software
*   Startup
*   Education
*   Retail
*   Automotive
*   Non-Profit

At Brain Station 23, we are passionate about delivering solutions that drive impact and growth.

## Join Our Team

If you're looking to join a team of passionate professionals who share your enthusiasm for innovation and IT services, check out our latest job openings:

[Learn More](link-to-careers-page)

By joining us, you'll have the opportunity to work on cutting-edge projects, collaborate with like-minded individuals, and contribute to the growth of innovative solutions that transform industries.

We look forward to welcoming talented professionals to our team!

## Contact Us

Get in touch with us today:

Email: [careers@brainstation23.com](mailto:careers@brainstation23.com)

Phone: UAE +971 42420223, USA +1 201 534 7200, Malaysia +601111708999

In [27]:
def stream_brochure(company_name, url):
    stream = ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [28]:
stream_brochure("Brain Station 23", "https://brainstation-23.com/?bs")

Found links: {'links': [{'type': 'about page', 'url': 'https://brainstation-23.com/about/'}, {'type': 'site', 'url': 'https://brainstation-23.com/'}, {'type': 'services page', 'url': 'https://brainstation-23.com/services/'}, {'type': 'field force automation page', 'url': 'https://brainstation-23.com/field-force-automation/'}, {'type': 'adobe experience manager AEM page', 'url': 'https://brainstation-23.com/adobe-experience-manager-aem/'}, {'type': 'banking solution page', 'url': 'https://brainstation-23.com/banking-solution/'}, {'type': 'mobile app development page', 'url': 'https://brainstation-23.com/mobile-app-development/'}, {'type': 'augmented reality page', 'url': 'https://brainstation-23.com/augmented-reality/'}, {'type': 'virtual reality page', 'url': 'https://brainstation-23.com/virtual-reality/'}, {'type': 'industries page', 'url': 'https://brainstation-23.com/industries/'}, {'type': 'fintech solutions page', 'url': 'https://brainstation-23.com/fintech-solutions/'}, {'type': 

**Brain Station 23: Empowering Businesses with Innovative IT Solutions**

Welcome to Brain Station 23, a leading provider of IT services and solutions that empower businesses to thrive in today's digital landscape. With over 17 years of experience serving enterprises and SMEs worldwide, we have established ourselves as a trusted partner for organizations seeking cutting-edge technology and expertise.

**Our Mission**
At Brain Station 23, our mission is to deliver tailored digital transformation solutions and advanced IT services that help businesses unlock their full potential. We achieve this by fostering a culture of innovation, collaboration, and excellence within our team of highly skilled professionals.

**Our Culture**

* **Innovation**: We encourage creativity and innovation in everything we do, from developing new technologies to finding innovative solutions for our clients' problems.
* **Collaboration**: Our diverse team of experts works closely together to deliver exceptional results and support each other's growth and development.
* **Excellence**: We strive for excellence in every aspect of our work, from the quality of our services to the way we communicate with our clients and partners.

**Our Expertise**

* **Industry-specific solutions**: We offer IT services tailored to the specific needs of industries such as Fintech, Healthcare, Telecom, Real Estate, Software, and Startup.
* **Cloud & Cyber Security**: Our experts provide comprehensive cloud and cybersecurity solutions to protect businesses from emerging threats.
* **Data Science & Business Intelligence**: We help organizations make data-driven decisions using advanced analytics and data visualization tools.

**Our Clients**

* **Key Client Portfolio**: Brain Station 23 serves a global client base across multiple industries, including Fintech, Healthcare, Telecom, Real Estate, Software, and Startup.
* **Global Family**: Our USA office delivers tailored digital transformation solutions and advanced IT services worldwide, while our Dubai office is the central hub for connecting our global operations.

**Join Our Team**

If you're passionate about innovation, collaboration, and excellence, we invite you to join us as a professional partner. Visit our Careers page to explore job opportunities or contact us directly to learn more about joining our global family.

**Stay Connected**

Follow us on social media:

* Facebook: [www.facebook.com/BrainStation23](http://www.facebook.com/BrainStation23)
* LinkedIn: [www.linkedin.com/company/brain-station-23](http://www.linkedin.com/company/brain-station-23)
* Twitter: [www.twitter.com/BST_UMX](http://www.twitter.com/BST_UMX)
* YouTube: [www.youtube.com/BrainStation23](http://www.youtube.com/BrainStation23)

Learn more about Brain Station 23 and how we can help your business thrive. Contact us today!